In [1]:
%%html
<style>
.formasaurus-field-highlighted { outline: 2px solid magenta;}
</style>

In [1]:
import re

from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import HTML, display

from formasaurus.html import load_html, get_cleaned_form_html, get_field_names, get_fields_to_annotate
from formasaurus.annotation import print_form_html
from formasaurus.storage import Storage
from formasaurus.widgets import (
    HtmlView, FormTypeSelect, FieldTypeSelect, FormAnnotator, MultiFormAnnotator, 
)
from formasaurus.utils import inverse_mapping
from formasaurus import annotation

In [3]:
storage = Storage("../formasaurus/data/")
index = storage.get_index()

def save_func():
    storage.write_index(index)    
    
field_schema = storage.get_field_schema()
form_schema = storage.get_form_schema()
all_annotations = list(storage.iter_annotations(index, drop_na=False))

def ann_by_type(types):
    """ Return all annotations with given form types (short names) """
    return [a for a in all_annotations if a.type in types]


def ann_by_field_type(field_type):
    """ Return all annotations which has at least one field of a given type """
    return [
        a for a in all_annotations 
        if field_type in a.info['visible_html_fields'][a.index].values()
   ]    

def get_fields_annotation(ann):
    return ann.info['visible_html_fields'][ann.index]

def fields_annotation_complete(ann, na_value):
    field_ann = get_fields_annotation(ann)
    if not field_ann:
        return True
    return all(v != na_value for v in field_ann.values())

def fields_annotation_partial(ann, na_value):
    field_ann = get_fields_annotation(ann)
    if not field_ann:
        return False
    values = field_ann.values()
    return any(v == na_value for v in values) and not all(v == na_value for v in values)    

In [4]:
storage.get_form_type_counts()

Counter({'N': 9,
         'P': 6,
         'V': 16,
         'b': 22,
         'c': 91,
         'l': 185,
         'm': 78,
         'o': 77,
         'p': 78,
         'r': 123,
         's': 287})

In [5]:
partial_annotations = [a for a in all_annotations if fields_annotation_partial(a, field_schema.na_value)]
not_annotated_annotations = [
    a for a in all_annotations if not fields_annotation_complete(a, field_schema.na_value)
]
_annotations = partial_annotations + not_annotated_annotations
MultiFormAnnotator(_annotations, form_schema.types, field_schema.types, annotate_types=True, save_func=save_func)

In [7]:
all_annotations[0]

FormAnnotation(form=<Element form at 0x110a58048>, type='l', index=0, info={'url': 'http://www.acozinhar.pt/portal/parceiros/', 'visible_html_fields': [{'pwd': 'p1', 'log': 'us', 'submit': 'Bs', 'rememberme': 'rC'}], 'forms': ['l']}, key='html/www.acozinhar.pt-0.html')

In [7]:
_annotations = ann_by_field_type('ab')
MultiFormAnnotator(_annotations, form_schema.types, field_schema.types, 
                   annotate_types=True, annotate_fields=True, save_func=save_func)

In [21]:
# _annotations = [a for a in all_annotations if a.type == 'b']
# MultiFormAnnotator(form_na_annotations, form_types, field_types, annotate_types=True, save_func=save_func)